# Propuesta de modelo

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

In [98]:
# Lectura de datos
data_entre = pd.read_csv(r"C:\Users\danie\Dengue\Dengue_fill\Datos\Dengue_entre.csv", dtype={36: str}, low_memory=False)
data_test = pd.read_csv(r"C:\Users\danie\Dengue\Dengue_fill\Datos\Dengue_test.csv", dtype={36: str}, low_memory=False)


# Correcion de tipo a DateTime de variables tipo fecha
columnas_fecha = ["FEC_CON", "INI_SIN"]
data[columnas_fecha] = data[columnas_fecha].apply(pd.to_datetime, errors='coerce', dayfirst = True)

#Seleccion de variables de estudio"
columnas = ["EDAD_AJUSTADA", "SEXO", "SEMANA", 
            "EVENTO", "ESTADO_FINAL", 
            "PAC_HOS", "CON_FIN", "confirmados",
            "FEC_CON", "INI_SIN",
           ]

df_entre = data_entre[columnas]
df_test = data_test[columnas]

In [104]:
# Reemplazo de NA
mediana_est = df_entre['ESTADO_FINAL'].median()
df_entre['ESTADO_FINAL'] = df_entre['ESTADO_FINAL'].fillna(mediana_est)

mediana_eda = df_entre['EDAD_AJUSTADA'].median()
df_entre['EDAD_AJUSTADA'] = df_entre['EDAD_AJUSTADA'].fillna(mediana_eda)

df_entre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500444 entries, 0 to 500443
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   EDAD_AJUSTADA  500444 non-null  float64       
 1   SEXO           500444 non-null  object        
 2   SEMANA         500444 non-null  int64         
 3   EVENTO         500444 non-null  object        
 4   ESTADO_FINAL   500444 non-null  float64       
 5   PAC_HOS        500444 non-null  int64         
 6   CON_FIN        500444 non-null  int64         
 7   confirmados    500444 non-null  int64         
 8   FEC_CON        500352 non-null  datetime64[ns]
 9   INI_SIN        500391 non-null  datetime64[ns]
 10  FEC_DIF        500444 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(4), object(2)
memory usage: 42.0+ MB


In [105]:
def calculo_fechas (df, col_inicio = 'INI_SIN', col_fin = 'FEC_CON', umbral=60):
    """
    Calcular la diferencia entre fechas
    """
    df = df.copy() 
    df[col_inicio] = pd.to_datetime(df[col_inicio], errors='coerce', dayfirst=True)
    df[col_fin] = pd.to_datetime(df[col_fin], errors='coerce', dayfirst=True)

    df['FEC_DIF'] = (df[col_fin] - df[col_inicio]).dt.days
    df['FEC_DIF'] = pd.to_numeric(df['FEC_DIF'], errors='coerce')
    
    df.loc[df['FEC_DIF'] > umbral, 'FEC_DIF'] = np.nan
    
    mediana = df['FEC_DIF'].median()
    df['FEC_DIF'] = df['FEC_DIF'].fillna(mediana)

    return df

df_entre = calculo_fechas(df_entre)
df_test = calculo_fechas(df_test)

## Construcción del Modelo
Modelo de regresión logística:

**Variable respuesta:** confirmado\
**Variables predictoras:** EDAD_AJUSTADA, PAC_HOS, SEXO, EVENTO, SEMANA, CON_FIN, EVENTO_FINAL, FEC_DIF.


Se propone la construcción de:
- Modelo de regresión logística
- Modelo de clasificación (Arboles de clasificación / Random Forest)

La intención es clasificar casos para determinar si serán confirmados cono Dengue o no, dependiendo de las variables de estudio.

In [113]:
#Primer intento del modelo

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 2. Definir variables predictoras (X) y variable objetivo (y)
columnas_predictoras = ["EDAD_AJUSTADA", "SEXO", "SEMANA", "EVENTO", 
                        "PAC_HOS", "CON_FIN", "FEC_DIF", "ESTADO_FINAL"]
X_train = df_entre[columnas_predictoras]
X_test = df_test[columnas_predictoras]

# Variable objetivo (suponiendo que "ESTADO_FINAL" es el resultado binario)
y_train = df_entre["confirmados"]
y_test = df_test["confirmados"]

# Convertir variables categóricas en dummies
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Normalizar variables numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo logístico
modelo = LogisticRegression(max_iter=500, solver="liblinear")  
modelo.fit(X_train_scaled, y_train)

# Predicción
y_pred = modelo.predict(X_test_scaled)

# Evaluación del modelo
print("🔹 Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\n🔹 Reporte de clasificación:")
print(classification_report(y_test, y_pred))
print(f"\n🔹 Precisión del modelo: {accuracy_score(y_test, y_pred):.4f}")



🔹 Matriz de confusión:
[[140216    428]
 [    35 306157]]

🔹 Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    140644
           1       1.00      1.00      1.00    306192

    accuracy                           1.00    446836
   macro avg       1.00      1.00      1.00    446836
weighted avg       1.00      1.00      1.00    446836


🔹 Precisión del modelo: 0.9990
